#### Keras 分类测试

In [1]:
import sys
sys.path.append("..")

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.callbacks import EarlyStopping
from stock_ai import ploter
from examples import get_data_classification

Using TensorFlow backend.


PLEASE install TALIB to call these methods
PLEASE install TALIB to call these methods


#### 获取数据

完整数据为601398日线数据+399300日线数据

### 测试对第2日的结果进行分类

默认取 5 天的数据作为输入数据。第 6 天的数据为结果数据。以此类推。采用 `TimeseriesGenerator` 对数据进行拆分。

In [2]:
(x_train, y_train), (x_test, y_test) = get_data_classification()

print(x_train.shape, 'x_train sequences')
print(y_train.shape, 'y_train sequences')
print(x_test.shape, 'x_test sequences')
print(y_test.shape, 'y_test sequences')

(2679, 5, 12) x_train sequences
(2679, 19) y_train sequences
(298, 5, 12) x_test sequences
(298, 19) y_test sequences


`x` 为所有数据（标准化后）标准化采用的是 `sklearn.preprocessing.MinMaxScaler`

In [3]:
x_train[0]

array([[7.79060181e-02, 3.57777936e-02, 7.00826446e-02, 8.08298532e-04,
        9.21014981e-02, 8.92299908e-02, 4.00000000e-01, 2.22222222e-01,
        2.85714286e-01, 1.11111111e-01, 1.87297409e-01, 1.85802948e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.22176422e-02, 2.83732840e-01, 5.94214876e-02, 3.12272666e-01,
        2.58381748e-01, 2.73364733e-01, 2.00000000e-01, 5.55555556e-01,
        2.85714286e-01, 5.55555556e-01, 2.29311511e-01, 2.32559399e-01],
       [7.03874691e-01, 8.69194111e-01, 6.61983471e-01, 7.55489694e-01,
        1.00000000e+00, 1.00000000e+00, 4.00000000e-01, 8.88888889e-01,
        4.28571429e-01, 5.55555556e-01, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        6.70516248e-01, 6.51765760e-01, 1.00000000e+00, 1.00

`y`为601398的 close.pct_change() 后的数据（onehot编码后）。onehot编码采用 `sklearn.preprocessing.OneHotEncoder`

>> 默认 `y` 值保留2位小数。（也就是说正常情况下计算精度在 `-10%~10%` 之间，数据间隔为1%）

In [4]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [5]:
def build_model():
    print('Building model...')
    model = Sequential()
    model.add(LSTM(512, input_shape=x_train[0].shape))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']
                 )
    return model

In [7]:
batch_size = 32
epochs = 1000
num_classes = y_train.shape[1]

model=build_model()

history = model.fit(x_train,
                    y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_split=0.1,
                    callbacks=[EarlyStopping()])

Building model...
Train on 2411 samples, validate on 268 samples
Epoch 1/1000
 - 7s - loss: 1.9577 - acc: 0.3484 - val_loss: 1.8910 - val_acc: 0.3955
Epoch 2/1000
 - 5s - loss: 1.8388 - acc: 0.3633 - val_loss: 1.9063 - val_acc: 0.3955


In [8]:
keras.utils.print_summary(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 512)               1075200   
_________________________________________________________________
activation_3 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 19)                9747      
_________________________________________________________________
activation_4 (Activation)    (None, 19)                0         
Total params: 1,084,947
Trainable params: 1,084,947
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ploter.plot_keras_history(history)

In [9]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
'Test score:{0},Test accuracy:{1}'.format(score[0],score[1])

298/298 [==============================] - ETA:  - ETA:  - ETA:  - 0s 576us/step


'Test score:1.7843725777312414,Test accuracy:0.3791946312725144'

### 测试对第5日的结果进行分类

In [10]:
(x_train, y_train), (x_test, y_test) = get_data_classification(skip_Y=5)
batch_size = 32
epochs = 1000
num_classes = y_train.shape[1]

model=build_model()

history = model.fit(x_train,
                    y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_split=0.1,
                    callbacks=[EarlyStopping()])
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
'Test score:{0}\nTest accuracy:{1}'.format(score[0],score[1])

Building model...
Train on 2406 samples, validate on 268 samples
Epoch 1/1000
 - 6s - loss: 2.0190 - acc: 0.3533 - val_loss: 1.8459 - val_acc: 0.4216
Epoch 2/1000
 - 5s - loss: 1.8830 - acc: 0.3716 - val_loss: 1.8501 - val_acc: 0.4216
298/298 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 699us/step


'Test score:1.8505963215091885\nTest accuracy:0.37919463087248323'